### Aim of the notebook is to visualize the receptive fields of convolution filters 

This notebook is simplified version of github repo [https://github.com/metalbubble/cnnvisualizer](https://github.com/metalbubble/cnnvisualizer)

Parts of codes have been used for the ICLR'15 paper Object Detectors Emerge in Deep Scene CNN

In [2]:
import torch
from torch.autograd import Variable as V
from torchvision import transforms as trn
import os
import numpy as np
import cv2
from dataset import Dataset
import torchvision.models as models
import torch.utils.data as data
import torchvision.models as models

In [3]:
# visualization setup
img_size = (224, 224)       # input image size
segment_size = (120,120)    # the unit segmentaiton size
num_top = 12                # how many top activated images to extract
margin = 3                  # pixels between two segments
threshold_scale = 0.2       # the scale used to segment the feature map. Smaller the segmentation will be tighter.

In [4]:
# dataset setup
batch_size = 64
num_workers = 6

In [5]:
# Load required model
model_name = 'resnet18_imagenet'
model = models.resnet18(pretrained=True)
features_names = ['layer4']

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/marco/.torch/models/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [01:18<00:00, 593709.51it/s]


In [7]:
model.eval()
#model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [8]:
### Download the image dataset
!sh download_images.sh

--2018-12-03 13:48:25--  http://places.csail.mit.edu/unit_annotation/data/images.tar
Resolving places.csail.mit.edu (places.csail.mit.edu)... 128.30.100.255
Connecting to places.csail.mit.edu (places.csail.mit.edu)|128.30.100.255|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 755445760 (720M) [application/x-tar]
Saving to: ‘images.tar’

images.tar          100%[===================>] 720,45M   338KB/s    in 29m 21s 

2018-12-03 14:17:46 (419 KB/s) - ‘images.tar’ saved [755445760/755445760]

images/
images/readme.txt
images/imagenet/
images/imagenet/012988.jpg
images/imagenet/009763.jpg
images/imagenet/005704.jpg
images/imagenet/017503.jpg
images/imagenet/017971.jpg
images/imagenet/020938.jpg
images/imagenet/004144.jpg
images/imagenet/016896.jpg
images/imagenet/008123.jpg
images/imagenet/008684.jpg
images/imagenet/016343.jpg
images/imagenet/024454.jpg
images/imagenet/024826.jpg
images/imagenet/007051.jpg
images/imagenet/019596.jpg
images/imagenet/015983.jpg
imag

images/imagenet/006422.jpg
images/imagenet/014182.jpg
images/imagenet/006385.jpg
images/imagenet/018642.jpg
images/imagenet/014625.jpg
images/imagenet/000269.jpg
images/imagenet/005537.jpg
images/imagenet/005945.jpg
images/imagenet/009922.jpg
images/imagenet/009550.jpg
images/imagenet/017097.jpg
images/imagenet/005290.jpg
images/imagenet/017730.jpg
images/imagenet/020779.jpg
images/imagenet/008310.jpg
images/imagenet/013189.jpg
images/imagenet/004377.jpg
images/imagenet/001429.jpg
images/imagenet/021139.jpg
images/imagenet/016170.jpg
images/imagenet/024667.jpg
images/imagenet/006043.jpg
images/imagenet/018584.jpg
images/imagenet/014991.jpg
images/imagenet/018223.jpg
images/imagenet/014244.jpg
images/imagenet/011968.jpg
images/imagenet/007603.jpg
images/imagenet/015404.jpg
images/imagenet/015876.jpg
images/imagenet/019811.jpg
images/imagenet/019463.jpg
images/imagenet/008771.jpg
images/imagenet/001048.jpg
images/imagenet/004716.jpg
images/imagenet/024206.jpg
images/imagenet/016963.jpg
i

images/imagenet/008652.jpg
images/imagenet/004635.jpg
images/imagenet/024325.jpg
images/imagenet/021809.jpg
images/imagenet/016840.jpg
images/imagenet/016432.jpg
images/imagenet/004192.jpg
images/imagenet/001452.jpg
images/imagenet/001820.jpg
images/imagenet/013655.jpg
images/imagenet/021142.jpg
images/imagenet/000212.jpg
images/imagenet/009959.jpg
images/imagenet/020702.jpg
images/imagenet/012015.jpg
images/imagenet/006459.jpg
images/imagenet/003307.jpg
images/imagenet/018639.jpg
images/imagenet/023617.jpg
images/imagenet/011100.jpg
images/imagenet/002935.jpg
images/imagenet/002547.jpg
images/imagenet/007219.jpg
images/imagenet/010740.jpg
images/imagenet/022057.jpg
images/imagenet/019079.jpg
images/imagenet/000673.jpg
images/imagenet/012806.jpg
images/imagenet/012474.jpg
images/imagenet/020363.jpg
images/imagenet/021284.jpg
images/imagenet/013593.jpg
images/imagenet/001033.jpg
images/imagenet/021951.jpg
images/imagenet/021523.jpg
images/imagenet/016918.jpg
images/imagenet/013234.jpg
i

images/imagenet/003197.jpg
images/imagenet/002070.jpg
images/imagenet/007089.jpg
images/imagenet/010277.jpg
images/imagenet/015529.jpg
images/imagenet/022912.jpg
images/imagenet/022560.jpg
images/imagenet/002549.jpg
images/imagenet/007217.jpg
images/imagenet/019077.jpg
images/imagenet/022059.jpg
images/imagenet/015010.jpg
images/imagenet/018190.jpg
images/imagenet/006457.jpg
images/imagenet/006825.jpg
images/imagenet/003309.jpg
images/imagenet/014650.jpg
images/imagenet/023619.jpg
images/imagenet/018637.jpg
images/imagenet/009525.jpg
images/imagenet/009957.jpg
images/imagenet/005930.jpg
images/imagenet/005542.jpg
images/imagenet/017745.jpg
images/imagenet/009282.jpg
images/imagenet/004302.jpg
images/imagenet/008365.jpg
images/imagenet/024612.jpg
images/imagenet/016105.jpg
images/imagenet/014596.jpg
images/imagenet/003768.jpg
images/imagenet/006036.jpg
images/imagenet/018983.jpg
images/imagenet/023278.jpg
images/imagenet/014231.jpg
images/imagenet/018256.jpg
images/imagenet/006791.jpg
i

images/imagenet/014773.jpg
images/imagenet/018714.jpg
images/imagenet/002818.jpg
images/imagenet/007334.jpg
images/imagenet/015694.jpg
images/imagenet/007493.jpg
images/imagenet/019154.jpg
images/imagenet/015133.jpg
images/imagenet/004221.jpg
images/imagenet/024096.jpg
images/imagenet/008246.jpg
images/imagenet/016781.jpg
images/imagenet/004586.jpg
images/imagenet/013778.jpg
images/imagenet/024731.jpg
images/imagenet/016026.jpg
images/imagenet/008993.jpg
images/imagenet/009406.jpg
images/imagenet/009874.jpg
images/imagenet/020188.jpg
images/imagenet/005813.jpg
images/imagenet/005461.jpg
images/imagenet/017666.jpg
images/imagenet/012138.jpg
images/imagenet/000498.jpg
images/imagenet/007755.jpg
images/imagenet/019292.jpg
images/imagenet/019535.jpg
images/imagenet/019947.jpg
images/imagenet/022969.jpg
images/imagenet/015920.jpg
images/imagenet/015552.jpg
images/imagenet/006115.jpg
images/imagenet/014312.jpg
images/imagenet/018375.jpg
images/imagenet/009067.jpg
images/imagenet/005000.jpg
i

...

images/sun/007595.jpg
images/sun/015035.jpg
images/sun/006472.jpg
images/sun/006800.jpg
images/sun/026162.jpg
images/sun/014675.jpg
images/sun/018612.jpg
images/sun/012799.jpg
images/sun/009500.jpg
images/sun/009972.jpg
images/sun/000239.jpg
images/sun/005915.jpg
images/sun/005567.jpg
images/sun/025077.jpg
images/sun/017760.jpg
images/sun/020729.jpg
images/sun/004327.jpg
images/sun/001479.jpg
images/sun/016687.jpg
images/sun/008340.jpg
images/sun/024190.jpg
images/sun/004480.jpg
images/sun/021169.jpg
images/sun/008895.jpg
images/sun/016120.jpg
images/sun/024637.jpg
images/sun/023988.jpg
images/sun/006013.jpg
images/sun/014214.jpg
images/sun/026971.jpg
images/sun/026503.jpg
images/sun/011938.jpg
images/sun/018273.jpg
images/sun/019394.jpg
images/sun/007653.jpg
images/sun/019433.jpg
images/sun/019841.jpg
images/sun/015826.jpg
images/sun/015454.jpg
images/sun/001018.jpg
images/sun/004746.jpg
images/sun/024983.jpg
images/sun/008721.jpg
images/sun/024256.jpg
images/sun/016541.jpg
images/sun

images/sun/009622.jpg
images/sun/025880.jpg
images/sun/005645.jpg
images/sun/017442.jpg
images/sun/017830.jpg
images/sun/020879.jpg
images/sun/009185.jpg
images/sun/025355.jpg
images/sun/004005.jpg
images/sun/008062.jpg
images/sun/024515.jpg
images/sun/024967.jpg
images/sun/016202.jpg
images/sun/007110.jpg
images/sun/019370.jpg
images/sun/010449.jpg
images/sun/015317.jpg
images/sun/026995.jpg
images/sun/018297.jpg
images/sun/006750.jpg
images/sun/014925.jpg
images/sun/014557.jpg
images/sun/011209.jpg
images/sun/026240.jpg
images/sun/018530.jpg
images/sun/018942.jpg
images/sun/006781.jpg
images/sun/018246.jpg
images/sun/014221.jpg
images/sun/023268.jpg
images/sun/026944.jpg
images/sun/026536.jpg
images/sun/018993.jpg
images/sun/006026.jpg
images/sun/003778.jpg
images/sun/014586.jpg
images/sun/026291.jpg
images/sun/022428.jpg
images/sun/015813.jpg
images/sun/015461.jpg
images/sun/019406.jpg
images/sun/019874.jpg
images/sun/010498.jpg
images/sun/002138.jpg
images/sun/007666.jpg
images/sun

images/sun/010395.jpg
images/sun/002192.jpg
images/sun/022325.jpg
images/sun/010840.jpg
images/sun/010432.jpg
images/sun/001720.jpg
images/sun/008019.jpg
images/sun/021597.jpg
images/sun/013280.jpg
images/sun/001087.jpg
images/sun/016279.jpg
images/sun/021230.jpg
images/sun/013527.jpg
images/sun/013955.jpg
images/sun/025489.jpg
images/sun/009659.jpg
images/sun/000160.jpg
images/sun/012367.jpg
images/sun/020470.jpg
images/sun/020802.jpg
images/sun/017439.jpg
images/sun/005199.jpg
images/sun/025458.jpg
images/sun/012411.jpg
images/sun/012863.jpg
images/sun/020306.jpg
images/sun/009688.jpg
images/sun/000616.jpg
images/sun/005148.jpg
images/sun/021546.jpg
images/sun/021934.jpg
images/sun/013251.jpg
images/sun/024218.jpg
images/sun/013984.jpg
images/sun/004708.jpg
images/sun/001056.jpg
images/sun/015868.jpg
images/sun/022821.jpg
images/sun/022453.jpg
images/sun/010344.jpg
images/sun/010891.jpg
images/sun/002143.jpg
images/sun/011976.jpg
images/sun/011504.jpg
images/sun/023213.jpg
images/sun

images/sun/024912.jpg
images/sun/024560.jpg
images/sun/013529.jpg
images/sun/001089.jpg
images/sun/007165.jpg
images/sun/015362.jpg
images/sun/019305.jpg
images/sun/006725.jpg
images/sun/026592.jpg
images/sun/014285.jpg
images/sun/006082.jpg
images/sun/018937.jpg
images/sun/018545.jpg
images/sun/026235.jpg
images/sun/014522.jpg
images/sun/014950.jpg
images/sun/023919.jpg
images/sun/011978.jpg
images/sun/026543.jpg
images/sun/026931.jpg
images/sun/014254.jpg
images/sun/018233.jpg
images/sun/014981.jpg
images/sun/018594.jpg
images/sun/006053.jpg
images/sun/019801.jpg
images/sun/019473.jpg
images/sun/015414.jpg
images/sun/015866.jpg
images/sun/007613.jpg
images/sun/021548.jpg
images/sun/016973.jpg
images/sun/016501.jpg
images/sun/024216.jpg
images/sun/004706.jpg
images/sun/001058.jpg
images/sun/008761.jpg
images/sun/025824.jpg
images/sun/025456.jpg
images/sun/017341.jpg
images/sun/009686.jpg
images/sun/020308.jpg
images/sun/009121.jpg
images/sun/017894.jpg
images/sun/000618.jpg
images/sun

images/sun/016881.jpg
images/sun/004153.jpg
images/sun/016354.jpg
images/sun/008693.jpg
images/sun/013878.jpg
images/sun/024831.jpg
images/sun/024443.jpg
images/sun/005713.jpg
images/sun/009774.jpg
images/sun/025203.jpg
images/sun/017966.jpg
images/sun/017514.jpg
images/sun/003158.jpg
images/sun/006606.jpg
images/sun/018814.jpg
images/sun/018466.jpg
images/sun/026316.jpg
images/sun/023448.jpg
images/sun/014401.jpg
images/sun/014873.jpg
images/sun/015994.jpg
images/sun/019581.jpg
images/sun/007046.jpg
images/sun/002718.jpg
images/sun/015241.jpg
images/sun/022208.jpg
images/sun/019226.jpg
images/sun/022096.jpg
images/sun/010781.jpg
images/sun/002221.jpg
images/sun/022731.jpg
images/sun/015778.jpg
images/sun/010026.jpg
images/sun/002586.jpg
images/sun/003813.jpg
images/sun/003461.jpg
images/sun/026188.jpg
images/sun/006498.jpg
images/sun/011666.jpg
images/sun/014138.jpg
images/sun/023171.jpg
images/sun/000574.jpg
images/sun/000906.jpg
images/sun/012773.jpg
images/sun/020064.jpg
images/sun

In [9]:
# image datasest to be processed
name_dataset = 'sun+imagenetaval'
root_image = 'images'
with open('images/imagelist.txt') as f:
    lines = f.readlines()
imglist = []
for line in lines:
    line = line.rstrip()
    imglist.append(root_image + '/' + line)

In [10]:
features_blobs = []
def hook_feature(module, input, output):
    # hook the feature extractor
    features_blobs.append(np.squeeze(output.data.cpu().numpy()))

for name in features_names:
    model._modules.get(name).register_forward_hook(hook_feature)

In [11]:
# image transformer
tf = trn.Compose([
        trn.Resize(img_size),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [12]:
dataset = Dataset(imglist, tf)
loader = data.DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False)

In [ ]:
# extract the max value activaiton for each image
imglist_results = []
maxfeatures = [None] * len(features_names)
num_batches = len(dataset) / batch_size
for batch_idx, (input, paths) in enumerate(loader):
    del features_blobs[:]
    print('%d / %d' % (batch_idx+1, num_batches))
    input = input.cuda()
    input_var = V(input, volatile=True)
    logit = model.forward(input_var)
    imglist_results = imglist_results + list(paths)
    if maxfeatures[0] is None:
        # initialize the feature variable
        for i, feat_batch in enumerate(features_blobs):
            size_features = (len(dataset), feat_batch.shape[1])
            maxfeatures[i] = np.zeros(size_features)
    start_idx = batch_idx*batch_size
    end_idx = min((batch_idx+1)*batch_size, len(dataset))
    for i, feat_batch in enumerate(features_blobs):
        maxfeatures[i][start_idx:end_idx] = np.max(np.max(feat_batch,3),2)

In [13]:
# generate the top activated images
output_folder = 'result_segments/%s' % model_name
if not os.path.exists(output_folder):
    os.makedirs(output_folder + '/image')

In [14]:
# output the html first
for layerID, layer in enumerate(features_names):
    file_html = os.path.join(output_folder, layer + '.html')
    with open(file_html, 'w') as f:
        num_units = maxfeatures[layerID].shape[1]
        lines_units = ['%s-unit%03d.jpg' % (layer, unitID) for unitID in range(num_units)]
        lines_units = ['unit%03d<br><img src="image/%s">'%(unitID, lines_units[unitID]) for unitID in range(num_units)]
        f.write('\n<br>'.join(lines_units))

NameError: name 'maxfeatures' is not defined

In [14]:
# generate the unit visualization
for layerID, layer in enumerate(features_names):
    num_units = maxfeatures[layerID].shape[1]
    imglist_sorted = []
    # load the top actiatied image list into one list
    for unitID in range(num_units):
        activations_unit = np.squeeze(maxfeatures[layerID][:, unitID])
        idx_sorted = np.argsort(activations_unit)[::-1]
        imglist_sorted += [imglist[item] for item in idx_sorted[:num_top]]

    # data loader for the top activated images
    loader_top = data.DataLoader(
        Dataset(imglist_sorted, tf),
        batch_size=num_top,
        num_workers=num_workers,
        shuffle=False)
    for unitID, (input, paths) in enumerate(loader_top):
        del features_blobs[:]
        print('%d / %d' % (unitID+1, num_units))
        input = input.cuda()
        input_var = V(input, volatile=True)
        logit = model.forward(input_var)
        feature_maps = features_blobs[layerID]

        images_input = input.cpu().numpy()
        max_value = 0
        output_unit = []
        for i in range(num_top):
            feature_map = feature_maps[i][unitID]
            if max_value == 0:
                max_value = np.max(feature_map)
            feature_map = feature_map / max_value
            mask = cv2.resize(feature_map, segment_size)
            mask[mask < threshold_scale] = 0.0 # binarize the mask
            mask[mask > threshold_scale] = 1.0

            img = cv2.imread(paths[i])
            img = cv2.resize(img, segment_size)
            img = cv2.normalize(img.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
            img_mask = np.multiply(img, mask[:,:, np.newaxis])
            img_mask = np.uint8(img_mask * 255)
            output_unit.append(img_mask)
            output_unit.append(np.uint8(np.ones((segment_size[0],margin,3))*255))
        montage_unit = np.concatenate(output_unit, axis=1)
        cv2.imwrite(os.path.join(output_folder, 'image', '%s-unit%03d.jpg'%(layer, unitID)), montage_unit)
print('done check results in ' + output_folder)

1 / 512
2 / 512
3 / 512
4 / 512
5 / 512
6 / 512
7 / 512
8 / 512


/home/saurabh/anaconda3/envs/py36torch/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


9 / 512
10 / 512
11 / 512
12 / 512
13 / 512
14 / 512
15 / 512
16 / 512
17 / 512
18 / 512
19 / 512
20 / 512
21 / 512
22 / 512
23 / 512
24 / 512
25 / 512
26 / 512
27 / 512
28 / 512
29 / 512
30 / 512
31 / 512
32 / 512
33 / 512
34 / 512
35 / 512
36 / 512
37 / 512
38 / 512
39 / 512
40 / 512
41 / 512
42 / 512
43 / 512
44 / 512
45 / 512
46 / 512
47 / 512
48 / 512
49 / 512
50 / 512
51 / 512
52 / 512
53 / 512
54 / 512
55 / 512
56 / 512
57 / 512
58 / 512
59 / 512
60 / 512
61 / 512
62 / 512
63 / 512
64 / 512
65 / 512
66 / 512
67 / 512
68 / 512
69 / 512
70 / 512
71 / 512
72 / 512
73 / 512
74 / 512
75 / 512
76 / 512
77 / 512
78 / 512
79 / 512
80 / 512
81 / 512
82 / 512
83 / 512
84 / 512
85 / 512
86 / 512
87 / 512
88 / 512
89 / 512
90 / 512
91 / 512
92 / 512
93 / 512
94 / 512
95 / 512
96 / 512
97 / 512
98 / 512
99 / 512
100 / 512
101 / 512
102 / 512
103 / 512
104 / 512
105 / 512
106 / 512
107 / 512
108 / 512
109 / 512
110 / 512
111 / 512
112 / 512
113 / 512
114 / 512
115 / 512
116 / 512
117 / 512
11